In [11]:
%pip install langchain langchain_openai --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
import requests
from bs4 import BeautifulSoup
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.utilities import DuckDuckGoSearchAPIWrapper
import json
from dotenv import load_dotenv
from langchain.globals import set_debug
from IPython.display import display, Markdown

load_dotenv()
set_debug(False)

In [2]:
DEPLOYMENT_NAME="gpt-35-turbo"
MODEL_NAME = "gpt-35-turbo"
API_VERSION="2024-02-15-preview"

RESULTS_PER_QUESTION = 3

SUMMARY_TEMPLATE = """{text} 
-----------
Using the above text, answer in short the following question: 
> {question}
-----------
if the question cannot be answered using the text, imply summarize the text. Include all factual information, numbers, stats etc if available."""  # noqa: E501

SUMMARY_PROMPT = ChatPromptTemplate.from_template(SUMMARY_TEMPLATE)

SEARCH_PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            "Write 3 google search queries to search online that form an "
            "objective opinion from the following: {question}\n"
            "You must respond with a list of strings in array: "
            '["query 1", "query 2", "query 3"].',
        ),
    ]
)

WRITER_SYSTEM_PROMPT = "You are an AI critical thinker research assistant. Your sole purpose is to write well written, critically acclaimed, objective and structured reports on given text."  # noqa: E501

RESEARCH_REPORT_TEMPLATE = """Information:
--------
{research_summary}
--------
Using the above information, answer the following question or topic: "{question}" in a detailed report -- \
The report should focus on the answer to the question, should be well structured, informative, \
in depth, with facts and numbers if available and a minimum of 1,200 words.
You should strive to write the report as long as you can using all relevant and necessary information provided.
You must write the report with markdown syntax.
You MUST determine your own concrete and valid opinion based on the given information. Do NOT deter to general and meaningless conclusions.
Write all used source urls at the end of the report, and make sure to not add duplicated sources, but only one reference for each.
You must write the report in apa format.
Please do your best, this is very important to my career."""

RESEARCH_REPORT_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", WRITER_SYSTEM_PROMPT),
        ("user", RESEARCH_REPORT_TEMPLATE),
    ]
)

In [3]:
ddg_search = DuckDuckGoSearchAPIWrapper()

def web_search(query: str, num_results: int = RESULTS_PER_QUESTION):
    results = ddg_search.results(query, num_results)
    return [r["link"] for r in results]

def scrape_text(url: str):
    # Send a GET request to the webpage
    try:
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the content of the request with BeautifulSoup
            soup = BeautifulSoup(response.text, "html.parser")

            # Extract all text from the webpage
            page_text = soup.get_text(separator=" ", strip=True)

            # Print the extracted text
            return page_text
        else:
            return f"Failed to retrieve the webpage: Status code {response.status_code}"
    except Exception as e:
        print(e)
        return f"Failed to retrieve the webpage: {e}"

def collapse_list_of_lists(list_of_lists):
    content = []
    for l in list_of_lists:
        content.append("\n\n".join(l))
    return "\n\n".join(content)

llm = AzureChatOpenAI(
    azure_deployment=DEPLOYMENT_NAME, 
    model_name=MODEL_NAME, 
    openai_api_version=API_VERSION,
    temperature=0)

scrape_and_summarize_chain = RunnablePassthrough.assign(
    summary = RunnablePassthrough.assign(
    text=lambda x: scrape_text(x["url"])[:10000]
) | SUMMARY_PROMPT | llm | StrOutputParser()
) | (lambda x: f"URL: {x['url']}\n\nSUMMARY: {x['summary']}")

web_search_chain = RunnablePassthrough.assign(
    urls = lambda x: web_search(x["question"])
) | (lambda x: [{"question": x["question"], "url": u} for u in x["urls"]]) | scrape_and_summarize_chain.map()

search_question_chain = SEARCH_PROMPT | llm | StrOutputParser() | json.loads

full_research_chain = search_question_chain | (lambda x: [{"question": q} for q in x]) | web_search_chain.map()

research_report_chain = RunnablePassthrough.assign(
    research_summary= full_research_chain | collapse_list_of_lists
) | RESEARCH_REPORT_PROMPT | llm | StrOutputParser()


In [4]:
result = research_report_chain.invoke({"question": "Generative AI insurance use cases"})
display(Markdown(result))

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


# Report on Generative AI Insurance Use Cases

## Introduction
Generative AI, also known as Generative Adversarial Networks (GANs), has been making significant strides in various industries, including insurance. This report aims to explore the use cases of generative AI in the insurance sector, highlighting its benefits, challenges, and future prospects.

## Benefits of Generative AI in Insurance
Generative AI offers several benefits when applied in the insurance industry. These benefits include:

### Claims Processing Automation
Generative AI can automate and streamline the claims processing workflow, leading to faster and more accurate claim settlements. By analyzing historical data and patterns, generative AI can assist in identifying fraudulent claims and processing legitimate claims efficiently.

### Product Development Assistance
By analyzing customer feedback and market trends, generative AI can provide valuable insights to insurance companies for developing new and innovative insurance products. This can lead to enhanced customer satisfaction and competitive advantage in the market.

### Regulatory Compliance
Generative AI can assist insurance companies in ensuring compliance with regulatory requirements. By analyzing vast amounts of data and identifying potential compliance issues, generative AI can help companies adhere to data protection laws, privacy regulations, and industry standards.

## Use Cases of Generative AI in Insurance
Generative AI can be applied to various use cases within the insurance industry, including:

### Customer Service and Support
Generative AI can be used to develop virtual assistants and chatbots that can provide personalized customer support, answer queries, and assist in policy management. This can lead to improved customer experience and operational efficiency.

### Policy Recommendations
By analyzing customer data and preferences, generative AI can provide personalized policy recommendations tailored to individual needs and risk profiles. This can enhance customer satisfaction and retention.

### Risk Prediction and Prevention
Generative AI can analyze historical data and patterns to predict and prevent potential risks, such as fraud, accidents, and natural disasters. This proactive approach can help insurance companies mitigate risks and minimize losses.

### Customer Retention and Loyalty
Generative AI can analyze customer behavior and preferences to develop targeted retention strategies. By offering personalized incentives and services, insurance companies can enhance customer loyalty and reduce churn rates.

## Challenges of Implementing Generative AI in Insurance
Despite its potential benefits, implementing generative AI in the insurance industry comes with several challenges, including:

### Data Quality and Availability
Ensuring the quality, relevance, and availability of data can be challenging due to incomplete or inaccurate data, data silos, and data privacy concerns. Insurance companies need to address these issues to leverage the full potential of generative AI.

### Regulatory Compliance
Compliance with various regulatory requirements, such as data protection laws, privacy regulations, and industry standards, is necessary when implementing generative AI solutions. Ensuring that generative AI models adhere to legal and ethical standards is crucial for successful implementation.

### Interpretability and Transparency
Generative AI models can be complex and difficult to interpret, leading to challenges in explaining decisions to regulators, customers, and other stakeholders. Ensuring the interpretability and transparency of generative AI models is essential for building trust and acceptance.

### Integration with Legacy Systems
Many insurance companies operate on legacy systems that may not be compatible with generative AI technologies. Integrating generative AI with existing systems can be complex and time-consuming, requiring careful planning and execution.

### Ethical and Bias Issues
Generative AI models can inherit biases from the data used to train them, leading to biased decisions and outcomes. Insurance companies need to address ethical and bias issues to ensure fairness and ethical use of generative AI in insurance.

## Future of Generative AI in Insurance
The future of generative AI in the insurance industry holds promising advancements, including:

### Risk Assessment
Generative AI can revolutionize risk assessment by analyzing vast amounts of data to predict and prevent potential risks. This can lead to more accurate underwriting and pricing strategies, ultimately benefiting both insurance companies and policyholders.

### Customized Insurance Products
Generative AI can enable the development of highly customized insurance products tailored to individual needs and preferences. This personalized approach can enhance customer satisfaction and loyalty.

### Data-Driven Decision Making
By leveraging generative AI, insurance companies can make data-driven decisions across various business functions, including underwriting, claims management, and customer service. This can lead to improved operational efficiency and strategic decision-making.

### Integration with Emerging Technologies
Generative AI can be integrated with emerging technologies, such as blockchain and Internet of Things (IoT), to create innovative insurance solutions. This integration can lead to enhanced security, transparency, and customer experience.

## Conclusion
Generative AI has the potential to transform the insurance industry by offering various use cases and benefits, including claims processing automation, product development assistance, and regulatory compliance. However, the challenges of implementing generative AI, such as data quality, regulatory compliance, interpretability, integration, and ethical issues, need to be carefully addressed. The future of generative AI in insurance holds promising advancements, including customized insurance products, data-driven decision making, and integration with emerging technologies.

## References
- Scaleupally. (n.d.). Generative AI in Insurance. Retrieved from https://scaleupally.io/blog/generative-ai-in-insurance/
- MarketsandMarkets. (n.d.). The Influence of Generative AI on the Insurance Industry. Retrieved from https://www.marketsandmarkets.com/industry-news/The-Influence-Of-Generative-AI-On-The-Insurance-Industry